# 编码器-解码器架构
:label:`sec_encoder-decoder`

在一般的序列到序列问题中，如机器翻译（:numref:`sec_machine_translation`），输入和输出的长度是可变且不对齐的。处理这类数据的标准方法是设计一个*编码器-解码器*架构（:numref:`fig_encoder_decoder`），该架构由两个主要组件组成：一个*编码器*，它接受可变长度的序列作为输入；以及一个*解码器*，它充当条件语言模型，接收编码后的输入和目标序列的左侧上下文，并预测目标序列中的下一个词元。

![编码器-解码器架构。](../img/encoder-decoder.svg)
:label:`fig_encoder_decoder`

以从英语到法语的机器翻译为例。给定一个英语输入序列：“They”，“are”，“watching”，“。”，这个编码器-解码器架构首先将可变长度的输入编码成一个状态，然后对该状态进行解码，逐个词元生成翻译后的序列作为输出：“Ils”，“regardent”，“。”。由于编码器-解码器架构构成了后续章节中不同序列到序列模型的基础，本节将把这种架构转换为将在后面实现的接口。

In [1]:
from torch import nn
from d2l import torch as d2l

## (**Encoder**)

在编码器接口中，
我们只是指定了
编码器接受变长序列作为输入 `X`。
具体的实现将由任何继承这个基础 `Encoder` 类的模型提供。

In [2]:
class Encoder(nn.Module):  #@save
    """The base encoder interface for the encoder--decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def forward(self, X, *args):
        raise NotImplementedError

## [**解码器**]

在以下解码器接口中，
我们添加了一个额外的`init_state`方法
将编码器输出（`enc_all_outputs`）
转换为编码状态。
请注意，这一步骤
可能需要额外的输入，
例如输入的有效长度，
这一点在:numref:`sec_machine_translation`中已经解释过。
为了逐个生成可变长度的序列，
每次解码器可能会将一个输入
（例如，上一个时间步生成的令牌）
和编码状态
映射到当前时间步的输出令牌。

In [3]:
class Decoder(nn.Module):  #@save
    """The base decoder interface for the encoder--decoder architecture."""
    def __init__(self):
        super().__init__()

    # Later there can be additional arguments (e.g., length excluding padding)
    def init_state(self, enc_all_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

## [**将编码器和解码器结合在一起**]

在前向传播中，
编码器的输出
被用来生成编码状态，
这个状态将进一步被
解码器用作其输入之一。

In [4]:
class EncoderDecoder(d2l.Classifier):  #@save
    """The base class for the encoder--decoder architecture."""
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_all_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_all_outputs, *args)
        # Return decoder output only
        return self.decoder(dec_X, dec_state)[0]

在下一节中，
我们将看到如何应用RNN来设计
基于这种编码器-解码器架构的
序列到序列模型。


## 摘要

编码器-解码器架构
可以处理输入和输出
都由可变长度序列组成的问题，
因此适用于序列到序列的问题，
如机器翻译。
编码器将一个可变长度的序列作为输入
并将其转换为具有固定形状的状态。
解码器将具有固定形状的编码状态
映射到一个可变长度的序列。


## 练习

1. 假设我们使用神经网络实现编码器-解码器架构。编码器和解码器必须是同一种类型的神经网络吗？
1. 除了机器翻译，你还能想到另一种可以应用编码器-解码器架构的应用吗？

[讨论](https://discuss.d2l.ai/t/1061)